In [ ]:
import io
import base64
import glob
import os
import cv2
import sys
import importlib
import torchvision
import matplotlib.pyplot as plt

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable
from ptcap.trainers import DataParallelWrapper
from ptcap.grad_cam_videos import GradCam
from ptcap.data.annotation_parser import V2Parser

sys.path.insert(0, "../") 

## Load Model

In [ ]:
from ptcap.model.captioners import EncoderDecoder
from ptcap.model.encoders import C3dLSTMEncoder
from ptcap.model.two_stream_encoders import TwoStreamEncoder
from ptcap.model.external_encoders import FCEncoder, JesterEncoder, BIJesterEncoder
from ptcap.model.decoders import LSTMDecoder, CoupledLSTMDecoder

net = EncoderDecoder(
        encoder=TwoStreamEncoder,
        decoder=CoupledLSTMDecoder,
        encoder_kwargs={"encoder_output_size": 1024,"c2d_out_ch": 32,
                   "c3d_out_ch": 32, "rnn_output_size":1024},#, "pretrained_path": "/home/farzaneh/PycharmProjects/pretrained_nets/fully_conv_net_on_smtsmt_20170627/model.checkpoint"},
        decoder_kwargs={"embedding_size": 256, "hidden_size": 1024, "num_lstm_layers": 2, 
        "vocab_size": 2728, "num_step" :17, "fc_size":1024}, 
        gpus=[0]).cuda()
net = DataParallelWrapper(net, device_ids=[0]).cuda(0)

In [ ]:
# path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_balanced_tokens/'
path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/ECCV/v2_gulp160_two_stream_c2_32_c3_32_labels_cutoff5_cassif1_cap0/'

checkpoint = torch.load(path + '/model.best')


net.load_state_dict(checkpoint["model"])
#checkpoint["model"].keys()

## Load Data

In [ ]:
validation_path = "/data/20bn-something-something-v2/v2-validation.json"
videos_folder = "/data-ssd/v2-gulp-160/"
caption_type = "label"

validation_parser = V2Parser(validation_path,
                                            videos_folder,
                               caption_type=caption_type)

In [ ]:
def path_generator(annotation_path, root_path):
    with open(annotation_path, "rt") as f:
        annotations = json.load(f)
    files = validation_parser.get_video_paths()
    labels = validation_parser.get_captions()
    return ((os.path.join(root_path, f), label) for f, label in zip(files, labels))
path_gen = path_generator('/data/20bn-something-something-v2/v2-train.json', 
                          '/data/20bn-something-something-v2/')

In [ ]:
from rtorchn.data.fileio import MpegReader

reader = MpegReader(12, (128, 128), keep_aspect_ratio=False)

## Preprocessing

In [ ]:
from rtorchn.data.preprocessing import default_evaluation_preprocesser

preprocessor = default_evaluation_preprocesser([48, 96, 96], 64.)

## Tokenizer

In [ ]:
from ptcap.data.tokenizer import Tokenizer

USER_MAXLEN=17
tokenizer = Tokenizer(user_maxlen=USER_MAXLEN)
tokenizer.load_dictionaries(path)

## Model Layers

In [ ]:
def run_model(net):
    gradients = []


    conv1_feats = net.module.encoder.c3d_extractor.conv1(video)
    pool1_feats = net.module.encoder.c3d_extractor.pool1(conv1_feats)

    conv2_feats = net.module.encoder.c3d_extractor.conv2(pool1_feats)
    pool2_feats = net.module.encoder.c3d_extractor.pool2(conv2_feats)

    conv3_feats = net.module.encoder.c3d_extractor.conv3(pool2_feats)  

    pool3_feats = net.module.encoder.c3d_extractor.pool3(conv3_feats)  

    conv4_feats = net.module.encoder.c3d_extractor.conv4(pool3_feats) 
    
    
    
    conv5_feats = net.module.encoder.c3d_extractor.conv5(conv4_feats)   
    conv6_feats = net.module.encoder.c3d_extractor.conv6(conv5_feats)   
    ## REGISTER HOOK ##
    conv6_feats.register_hook(lambda grad:gradients.append(grad))
    
    pool4_feats = net.module.encoder.c3d_extractor.pool4(conv6_feats)   

    h = pool4_feats.view(pool4_feats.size()[0:3])   
    h = h.permute(0, 2, 1)   
    
    h2 = net.module.encoder.c2d_extractor.extract_features(video)
    h = torch.cat([h, h2], 2)

    net.module.encoder.lstm.flatten_parameters()
    h, _ = net.module.encoder.lstm(h) 

    net_output = net.module.decoder(h, Variable(torch.zeros([1, 1])).long().cuda())
    return net_output, gradients, conv6_feats

## CAM calculating

In [1]:
def calc_cam(net, net_output, layer_feats, target_index, vocab_size, gradients, token_counter):

    one_hot = np.zeros((1, vocab_size), dtype=np.float32)
    one_hot[0][target_index] = 1
    one_hot = Variable(torch.from_numpy(one_hot), requires_grad=True)
    one_hot = torch.sum(one_hot.cuda() * net_output[:,token_counter,:])
    net.module.encoder.c2d_extractor.conv4.zero_grad()
    #net.module.classif_layer.zero_grad()
    one_hot.backward(retain_graph=True)
    
    grads_val = gradients[0].cpu().data.numpy() #take the only element out of gradients list
    feats_numpy = layer_feats.cpu().data.numpy()[0, :] #first sample of batch
    weights = np.mean(grads_val, axis=(2, 3, 4))[0, :]   #grads_val: (1, 256, 48, 10, 10)
    cam = np.ones(feats_numpy.shape[1:], dtype=np.float32)
    weights = weights / weights.max()
    for ii, ww in enumerate(weights):
        cam += ww * feats_numpy[ii, :, :, :]
#     cam = np.maximum(cam, 0)
    return cam

In [ ]:
path_to_video, label = next(path_gen)

video_uint8 = reader.open(path_to_video)
video = preprocessor(video_uint8)
video = Variable(torch.from_numpy(video[None]), volatile=False).cuda()
tokens = tokenizer.encode_caption(label)
vocab_size = 2728

print(label)
all_cams = []
for tt in range(0,1):

    target_index =  tokens[tt]
    
    ##model
    net_output, gradients, layer_feats = run_model(net)
    cam = calc_cam(net, net_output, layer_feats, target_index, vocab_size, gradients, tt)
    
    print(tokenizer.decode_caption([target_index]))
    print("*"*15)
    if target_index==1: #the token is <END>
       break
    for gg in range(0,video_uint8.shape[0]-7, 1):
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        single_cam = cam[gg]
        single_cam = cv2.resize(single_cam, (224, 224))
        single_cam = single_cam - np.min(single_cam)
        single_cam = single_cam / np.max(single_cam)
        plt.imshow(single_cam)
        plt.subplot(1, 2, 2)
        plt.imshow(video_uint8[gg])
        plt.show()
        all_cams.append(single_cam)

In [ ]:
cam.shape

In [ ]:
path_to_video, label = next(path_gen)

video_uint8 = reader.open(path_to_video)
video = preprocessor(video_uint8)
video = Variable(torch.from_numpy(video[None]), volatile=False).cuda()
tokens = tokenizer.encode_caption(label)
vocab_size = 2728

print(label)
all_cams = []
for gg in range(0,video_uint8.shape[0]-7, 4):
    cam = []
    for tt in range(0,len(tokens)-1):
        target_index =  tokens[tt]
        ##model
        net_output, gradients, layer_feats = run_model(net)
        single_cam = calc_cam(net, net_output, layer_feats, target_index, vocab_size, gradients, tt)[gg]
        single_cam = cv2.resize(single_cam, (224, 224))
        single_cam = single_cam - np.min(single_cam)
        single_cam = single_cam / np.max(single_cam)
        cam.append(single_cam)
#         print(tokenizer.decode_caption([target_index]))
#         print("*"*15)
    single_cam = np.mean(np.array(cam), axis=0)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    
    plt.imshow(single_cam)
    plt.subplot(1, 2, 2)
    plt.imshow(video_uint8[gg])
    plt.show()
    all_cams.append(single_cam)
    

In [ ]:
class UnNormalize(object):
    """Unnormalize an tensor image with mean and standard deviation.
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel x std) + mean
    Args:
        mean (sequence): Sequence of means for R, G, B channels respecitvely.
        std (sequence): Sequence of standard deviations for R, G, B channels
            respecitvely.
    """

    def __init__(self, mean, std):
        self.mean = np.array(mean).astype('float32')
        self.std = np.array(std).astype('float32')

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        if isinstance(tensor, torch.Tensor):
            self.mean = torch.FloatTensor(self.mean)
            self.std = torch.FloatTensor(self.std)

            if (self.std.dim() != tensor.dim() or
                    self.mean.dim() != tensor.dim()):
                for i in range(tensor.dim() - self.std.dim()):
                    self.std = self.std.unsqueeze(-1)
                    self.mean = self.mean.unsqueeze(-1)

            tensor = torch.add(torch.mul(tensor, self.std), self.mean)
        else:
            # Relying on Numpy broadcasting abilities
            tensor = tensor * self.std + self.mean
        return tensor

In [ ]:
unnormalize_op = UnNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_data_unnormalised = unnormalize_op(video.data.cpu().squeeze(0))
input_data_unnormalised = input_data_unnormalised.permute(1, 2, 3, 0).numpy()  # (16x224x224x3)
input_data_unnormalised = np.flip(input_data_unnormalised, 3)

output_images_folder_cam_combined = os.path.join("cam_saved_images", "/home/farzaneh/cam/", "combined")

output_images_folder_original = os.path.join("cam_saved_images", "/home/farzaneh/cam/", "original")
output_images_folder_cam = os.path.join("cam_saved_images", "/home/farzaneh/cam", "cam")

os.makedirs(output_images_folder_cam_combined, exist_ok=True)
os.makedirs(output_images_folder_cam, exist_ok=True)
os.makedirs(output_images_folder_original, exist_ok=True)

clip_size = len(all_cams)

RESIZE_SIZE = 96
RESIZE_FLAG = 1
SAVE_INDIVIDUALS = 1
for i in range(clip_size):
    input_data_img = input_data_unnormalised[i, :, :, :]
    heatmap = cv2.applyColorMap(np.uint8(255 * all_cams[i]), cv2.COLORMAP_JET)
    if RESIZE_FLAG:
        input_data_img = cv2.resize(input_data_img, (RESIZE_SIZE, RESIZE_SIZE))
        heatmap = cv2.resize(heatmap, (RESIZE_SIZE, RESIZE_SIZE))
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(input_data_img)
    cam = cam / np.max(cam)
    combined_img = np.concatenate((np.uint8(255 * input_data_img), np.uint8(255 * cam)), axis=1)
    
    cv2.imwrite(os.path.join(output_images_folder_cam_combined, "img%02d.jpg" % (i + 1)), combined_img)
    if SAVE_INDIVIDUALS:
        cv2.imwrite(os.path.join(output_images_folder_cam, "img%02d.jpg" % (i + 1)), np.uint8(255 * cam))
        cv2.imwrite(os.path.join(output_images_folder_original, "img%02d.jpg" % (i + 1)), np.uint8(255 * input_data_img))

In [ ]:
path_to_combined_gif = os.path.join(output_images_folder_cam_combined, "mygif.gif")
os.system("convert -delay 10 -loop 0 {}.jpg {}".format(
                                    os.path.join(output_images_folder_cam_combined, "*"),
                                    path_to_combined_gif))
